In [31]:
import requests
import openai
import faiss
import json
import numpy as np

from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

In [7]:
# Get documentation from OBP
obp_base_url = "https://test.openbankproject.com"
obp_version = "v5.1.0"

swagger_url = "{}/obp/v5.1.0/resource-docs/{}/swagger".format(obp_base_url, obp_version)
swagger_response = requests.get(swagger_url)
swagger_json = swagger_response.json()

In [8]:
def parse_swagger(swagger_json):
    endpoints = []
    for path, methods in swagger_json['paths'].items():
        for method, details in methods.items():
            endpoint_info = {
                'path': path,
                'method': method,
                'description': details.get('description', ''),
                'parameters': details.get('parameters', []),
                'responses': details.get('responses', {})
            }
            endpoints.append(endpoint_info)
    return endpoints

endpoints = parse_swagger(swagger_json)

In [28]:
# Create vector embeddings
def get_embeddings(texts):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    
    return [e.embedding for e in response.data]

In [29]:
texts = [f"{e['method'].upper()} {e['path']} - {e['description']}" for e in endpoints[0:10]]
embeddings = get_embeddings(texts)

In [32]:
# Convert embeddings to a numpy array
embeddings_np = np.array(embeddings).astype('float32')

# Create a FAISS index
index = faiss.IndexFlatL2(embeddings_np.shape[1])  # L2 distance index
index.add(embeddings_np)

# Optionally, save the index to disk for later use
faiss.write_index(index, 'endpoints_index.faiss')

# Save metadata for retrieval
with open('endpoints_metadata.json', 'w') as f:
    json.dump(endpoints, f)

In [ ]:
def search_endpoints(query):
    query_embedding = get_embeddings([query])[0]
    query_embedding_np = np.array([query_embedding]).astype('float32')

    
    
    # Perform the search
    distances, indices = index.search(query_embedding_np, k=5)
    
    # Load metadata
    with open('endpoints_metadata.json', 'r') as f:
        endpoints = json.load(f)
    
    # Retrieve matching endpoints
    matches = [endpoints[i] for i in indices[0]]
    return matches

